# ML with TensorFlow Extended (TFX) Pipelines on GCP
1. Extracting the new training data from the source using **BigQuery** [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen) component.
2. Validating new training data
    * Generating statistics from the the incoming data using [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) component.
    * Importing a fixed raw schema using [ImporterNode](https://github.com/tensorflow/tfx/blob/master/tfx/components/common_nodes/importer_node.py) component.
    * Validating data based on the schema using [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) component.
5. Transforming the data for ML using the [Transform](https://www.tensorflow.org/tfx/guide/transform) component on **Cloud Dataflow**.
6. Training the model using the [Trainer](https://www.tensorflow.org/tfx/guide/trainer) component **Cloud AI Platform Training**.
7. Evaluate the model using the [Evaluator](https://www.tensorflow.org/tfx/guide/evaluator) component.
8. Validate the model using a [Custom TFX](https://www.tensorflow.org/tfx/guide/custom_component) component.
9. Push the the blessed model to **Cloud AI Platform Prediction** using [Pusher](https://www.tensorflow.org/tfx/guide/pusher) component.
10. Query the [ML Metadata](https://www.tensorflow.org/tfx/guide/mlmd) in **Cloud SQL** metadb. 

This tutorial uses Google Cloud managed services (BigQuery, Dataflow, AI Platform, Cloud SQL) to run the TFX Pipeline.

### Prerequisite
1. Set the required IAM credentials.
2. Create Cloud Storage bucket for artifacts.
3. Upload data to BigQuery.
4. Create Cloud SQL instance for ML Metadata and setup [connection via proxy](https://cloud.google.com/sql/docs/mysql/connect-admin-proxy).

In [58]:
import os
import tensorflow.io.gfile as 

PROJECT_ID = 'ksalama-cloudml'
BUCKET_NAME = 'ksalama-cloudml-tfx'
REGION = 'europe-west1'
BQ_DATASET_NAME = 'tfx_samples'
BQ_TABLE_NAME = 'census'
SQL_INSTANCE_NAME = 'tfx-mlmd'
WORKSPACE = "gs://{}/{}".format(BUCKET_NAME, BQ_TABLE_NAME)
DF_TEMP = WORKSPACE + "/beam/tmp"

RAW_SCHEMA_DIR = '../raw_schema'
SRC_DIR = '../src'

# Training Pipeline

In [2]:
import tfx
import tensorflow as tf
print("Tensorflow Version:", tf.__version__)
print("TFX Version:", tfx.__version__)

Tensorflow Version: 1.15.0
TFX Version: 0.15.0


## 0. Create Interactive Context
This will use an ephemeral SQLite MLMD connection contained in the pipeline_root directory with file name "metadata.sqlite" will be used.

In [4]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

connection_config = tfx.orchestration.metadata.mysql_metadata_connection_config(
    host='127.0.0.1',
    port=3306,
    database='metadb',
    username='root',
    password='root'
)

PIPELINE_NAME = 'tfx-census-classification'

context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=WORKSPACE,
    metadata_connection_config=connection_config
)

### Create local and Dataflow Beam args

In [41]:
local_beam_pipeline_args = [
    '--runner=DirectRunner',
    '--project=' + PROJECT_ID,
    '--region=' + REGION,
]

df_beam_pipeline_args = [
    '--runner=DataflowRunner',
    '--project=' + PROJECT_ID,
    '--temp_location=' + DF_TEMP,
    '--staging_location=' + DF_TEMP,
    '--region=' + REGION,
]

## 1. Data Ingestion (ExampleGen)
1. Reads the CSV data files (expecting to include headers)
2. Split the data to train and eval sets
3. Write the data to TFRecords


* **Inputs**: ExternalPath
* **Ouptpus**: Examples (TFRecords)
* **Properties**: split ratio

In [16]:
from tfx.utils.dsl_utils import external_input
from tfx.proto import example_gen_pb2

output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))


query = """
    SELECT * 
    FROM 
    `{}.{}.{}`
""".format(PROJECT_ID, BQ_DATASET_NAME, BQ_TABLE_NAME)

In [24]:
example_gen = tfx.components.BigQueryExampleGen(
    query=query,
    instance_name='BQ_Data_Extraction_Spliting',
    output_config=output_config
)

context.run(
    example_gen, 
    beam_pipeline_args=local_beam_pipeline_args
)

ExecutionResult(
    component_id: BigQueryExampleGen.BQ_Data_Extraction_Spliting
    execution_id: 10
    outputs:
        examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: gs://ksalama-cloudml-tfx/census/BigQueryExampleGen.BQ_Data_Extraction_Spliting/examples/9/train/, split: train, id: 4)
            Artifact(type_name: ExamplesPath, uri: gs://ksalama-cloudml-tfx/census/BigQueryExampleGen.BQ_Data_Extraction_Spliting/examples/9/eval/, split: eval, id: 5)]
        ))

### Read sample of the extracted data...
Since TTF v1.15 is used, you need to enable eager execution. However, this causes problems to subsequent steps.

In [25]:
READ = False

if READ:
    tf.enable_eager_execution()

    import tensorflow_data_validation as tfdv

    train_uri = example_gen.outputs['examples'].get()[0].uri

    tfrecord_filenames = tf.data.Dataset.list_files(train_uri+"*")

    # Create a `TFRecordDataset` to read these files
    dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

    # Iterate over the first 3 records and decode them using a TFExampleDecoder
    for tfrecord in dataset.take(3):
        serialized_example = tfrecord.numpy()
        example = tfdv.TFExampleDecoder().decode(serialized_example)
        pprint(example)
        print("")

## 2. Data Validation
1. Generate the **statistics** for the data to validate.
2. Import the **raw_schema** created in the Data Analysis phase.
3. Validat the **statistics** against the schema and generate **anomalies** (if any).

### 2.1. Generating statistics for the data to validate (StatisticsGen)
* **Inputs**: Examples
* **Outputs**: ExampleStatistics
* **Properries**: None

In [27]:
statistics_gen = tfx.components.StatisticsGen(
    instance_name='Statistics_Generation',
    examples=example_gen.outputs['examples'])

context.run(
    statistics_gen, 
    beam_pipeline_args=local_beam_pipeline_args
)

ExecutionResult(
    component_id: StatisticsGen.Statistics_Generation
    execution_id: 12
    outputs:
        output: Channel(
            type_name: ExampleStatisticsPath
            artifacts: [Artifact(type_name: ExampleStatisticsPath, uri: gs://ksalama-cloudml-tfx/census/StatisticsGen.Statistics_Generation/output/12/train/, split: train, id: 6)
            Artifact(type_name: ExampleStatisticsPath, uri: gs://ksalama-cloudml-tfx/census/StatisticsGen.Statistics_Generation/output/12/eval/, split: eval, id: 7)]
        ))

In [28]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


### 2.2. Import the fixed raw schema (ImporterNode)
The **ImporterNode** allows you to import an external artifact to a component.
You need to specifiy:
1. Artifact Type
2. Artifcat Location

In [63]:
gcs_raw_schema_dir = WORKSPACE + '/raw_schema'
!gsutil cp -r {RAW_SCHEMA_DIR} {gcs_raw_schema_dir}

Copying file://../raw_schema/schema.pbtxt [Content-Type=application/octet-stream]...
/ [1 files][  5.1 KiB/  5.1 KiB]                                                
Operation completed over 1 objects/5.1 KiB.                                      


In [66]:
schema_importer = tfx.components.common_nodes.importer_node.ImporterNode(
    instance_name='Schema_Importer',
    source_uri=gcs_raw_schema_dir,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=True
)

context.run(schema_importer)

ExecutionResult(
    component_id: ImporterNode.Schema_Importer
    execution_id: 24
    outputs:
        result: Channel(
            type_name: SchemaPath
            artifacts: [Artifact(type_name: SchemaPath, uri: gs://ksalama-cloudml-tfx/census/raw_schema, split: , id: 14)]
        ))

In [67]:
context.show(schema_importer.outputs['result'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'capital_gain',INT,required,,-
'capital_loss',INT,required,,-
'education',STRING,required,,'education'
'education_num',INT,required,,-
'fnlwgt',INT,required,,-
'gender',STRING,required,,'gender'
'hours_per_week',INT,required,,-
'income_bracket',STRING,required,,'income_bracket'


,Values
Domain,
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'gender',"' Female', ' Male'"
'income_bracket',"' <=50K', ' >50K'"
'marital_status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'native_country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Holand-Netherlands', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia', 'Egypt'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"


### 2.3. Validate the input data statistics (ExampleValidator)
* **Inputs**: ExampleStatistics, Schema
* **Outputs**: ExampleAnomalies (if any)
* **Properties**: None

In [31]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result'],
    instance_name="Data_Validation"
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator.Data_Validation
    execution_id: 14
    outputs:
        output: Channel(
            type_name: ExampleValidationPath
            artifacts: [Artifact(type_name: ExampleValidationPath, uri: gs://ksalama-cloudml-tfx/census/ExampleValidator.Data_Validation/output/14/, split: , id: 9)]
        ))

In [32]:
context.show(example_validator.outputs['anomalies'])

## 3. Data Preprocessing

### 3.1. Implement the preprocessing logic

We need to implement the preprocessing logic in a python module: **transform.py**.

This module is expected to have **preprocessing_fn** method, which accept a dictionary of tensors representing the raw features), and return a dictionary of tensors representing the transformed features.

We use the **raw schema** to identify feature types and the required transformation.

The function is implemented using [TensorFlow Transform](https://www.tensorflow.org/tfx/guide/tft).

In [33]:
!mkdir {SRC_DIR}

mkdir: ../src: File exists


In [34]:
_transform_module_file = SRC_DIR + '/transform.py'

In [35]:
%%writefile {_transform_module_file}

import tensorflow_transform as tft
import tensorflow_data_validation as tfdv

TARGET_FEATURE_NAME = 'income_bracket'
WEIGHT_FEATURE_NAME = 'fnlwgt'
RAW_SCHEMA_LOCATION = '../raw_schema/schema.pbtxt'

raw_schema = tfdv.load_schema_text(RAW_SCHEMA_LOCATION)

def preprocessing_fn(input_features):

    processed_features = {}

    for feature in raw_schema.feature:
        
        # Pass the target feature as is.
        if feature.name in [TARGET_FEATURE_NAME, WEIGHT_FEATURE_NAME]:
            processed_features[feature.name] = input_features[feature.name]
            continue

        if feature.type == 1:
            # Extract vocabulary and integerize categorical features.
            processed_features[feature.name+"_integerized"] = tft.compute_and_apply_vocabulary(
                input_features[feature.name], vocab_filename=feature.name)
        else:
            # normalize numeric features.
            processed_features[feature.name+"_scaled"] = tft.scale_to_z_score(input_features[feature.name])

        # Bucketize age using quantiles. 
        quantiles = tft.quantiles(input_features["age"], num_buckets=5, epsilon=0.01)
        processed_features["age_bucketized"] = tft.apply_buckets(
            input_features["age"], bucket_boundaries=quantiles)

    return processed_features



Overwriting ../src/transform.py


### 3.2. Tranform train and eval data (Transform)

The component uses the transform output generated from transforming the train data to transform eval data.
That is, while the train data is **analyzed** and **transformed**, the eval data is **only transformed** uaing the output of the analyze phase (TransformGraph) on the train data.

* **Inputs**: train and eval data (Examples), raw schema (Schema), transformation module (file)
* **outputs**: transformed train and eval data (Examples), transform output (TransformGraph)

In [43]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=_transform_module_file,
    instance_name="Data_Transformation"
)

context.run(
    transform, 
    #beam_pipeline_args=df_beam_pipeline_args
)

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: gs://ksalama-cloudml-tfx/census/Transform.Data_Transformation/transform_output/18/.temp_path/tftransform_tmp/e8e96e81187a4bd7aa8abbb0854234a9/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: gs://ksalama-cloudml-tfx/census/Transform.Data_Transformation/transform_output/18/.temp_path/tftransform_tmp/984001a9a73b4c7d8effbefacd3ac6f1/saved_model.pb
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: gs://ksalama-cloudml-tfx/census/Transform.Data_Transformation/transform_output/18/.temp_path/tftransform_tmp/616780f463b44cca87f2336cd26435fb/assets
INFO:tensorflow:SavedModel written to: gs://ksalama-cloud

ExecutionResult(
    component_id: Transform.Data_Transformation
    execution_id: 18
    outputs:
        transform_output: Channel(
            type_name: TransformPath
            artifacts: [Artifact(type_name: TransformPath, uri: gs://ksalama-cloudml-tfx/census/Transform.Data_Transformation/transform_output/18/, split: , id: 10)]
        )
        transformed_examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: gs://ksalama-cloudml-tfx/census/Transform.Data_Transformation/transformed_examples/18/train/, split: train, id: 11)
            Artifact(type_name: ExamplesPath, uri: gs://ksalama-cloudml-tfx/census/Transform.Data_Transformation/transformed_examples/18/eval/, split: eval, id: 12)]
        ))

In [59]:
uri = transform.outputs['transform_graph'].get()[0].uri
gfile.listdir(uri)

['metadata/', 'transform_fn/', 'transformed_metadata/']

### Read sample of the transformed data
Since TTF v1.15 is used, you need to enable eager execution. However, this causes problems to subsequent steps.

In [48]:
READ = False

if READ:
    tf.enable_eager_execution()

    train_uri = transform.outputs['transformed_examples'].get()[1].uri
    tfrecord_filenames = [os.path.join(train_uri, name)
                          for name in os.listdir(train_uri)]
    dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
    decoder = tfdv.TFExampleDecoder()
    for tfrecord in dataset.take(3):
        serialized_example = tfrecord.numpy()
        example = decoder.decode(serialized_example)
        pprint(example)
        print("")

## 4. Train the Model (Trainer)

### 4.1 Implement a train Python module.

Create a Python module containing a **trainer_fn** function, which must return an estimator.

The **trainer_fn** receives the following parameters:
* hparams: currently includes train_steps, eval_steps, and transform_output.
* schema: the raw data schema. This is used to create the serving input function to export the model.

The **trainer_fn** returns a dictionary of the following:
* estimator: The estimator that will be used for training and eval.
* train_spec: Spec for training.
* eval_spec: Spec for eval.
* eval_input_receiver_fn: Input function for eval.

In [49]:
_train_module_file = SRC_DIR + '/train.py'

In [71]:
%%writefile {_train_module_file}

import math

import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_model_analysis as tfma
from tensorflow_transform.tf_metadata import schema_utils


TARGET_FEATURE_NAME = 'income_bracket'
WEIGHT_FEATURE_NAME = 'fnlwgt'
TARGET_FEATURE_LABELS = [' <=50K', ' >50K']

# Input function for train and eval transformed data
def make_input_fn(tfrecords_files, transform_output,
                  batch_size, num_epochs=1, shuffle=False):
    
    def _gzip_reader_fn(filenames):
        return tf.data.TFRecordDataset(
            filenames, compression_type='GZIP')
    
    def input_fn():
        dataset = tf.data.experimental.make_batched_features_dataset(
            file_pattern=tfrecords_files,
            batch_size=batch_size,
            features=transform_output.transformed_feature_spec(),
            label_key=TARGET_FEATURE_NAME,
            reader=_gzip_reader_fn,
            num_epochs=num_epochs,
            shuffle=shuffle
        )
        return dataset
    return input_fn


# Create estimator
def create_estimator(params, transform_output, run_config):
    
    feature_columns = []
    
    # Create feature columns based on the transform schema
    transformed_features = transform_output.transformed_metadata.schema.feature
    for feature in transformed_features:
        
        if feature.name in [TARGET_FEATURE_NAME, WEIGHT_FEATURE_NAME]:
            continue

        if hasattr(feature, 'int_domain') and feature.int_domain.is_categorical:
            vocab_size = feature.int_domain.max + 1
            feature_columns.append(
                tf.feature_column.embedding_column(
                    tf.feature_column.categorical_column_with_identity(
                        feature.name, num_buckets=vocab_size),
                    dimension = int(math.sqrt(vocab_size))))
        else:
            feature_columns.append(
                tf.feature_column.numeric_column(feature.name))
            
    # Create DNNClassifier        
    estimator = tf.estimator.DNNClassifier(
        weight_column=WEIGHT_FEATURE_NAME,
        label_vocabulary=TARGET_FEATURE_LABELS,
        feature_columns=feature_columns,
        hidden_units=params.hidden_units,
        warm_start_from=params.warm_start_from,
        config=run_config
    )
    
    return estimator


# Create serving input function for model serving
def _serving_input_receiver_fn(transform_output, schema):
    
    raw_feature_spec = schema_utils.schema_as_feature_spec(schema).feature_spec
    raw_feature_spec.pop(TARGET_FEATURE_NAME)
    raw_feature_spec.pop(WEIGHT_FEATURE_NAME)
    
#     # Create the interface for the serving function with the raw features
#     serving_input_receiver = tf.estimator.export.build_parsing_serving_input_receiver_fn(
#           raw_feature_spec, default_batch_size=None)()
    
#     # Apply the transform function 
#     transformed_features = transform_output.transform_raw_features(
#         serving_input_receiver.features)
    
#     return tf.estimator.export.ServingInputReceiver(
#         transformed_features, 
#         serving_input_receiver.features
#     )

    # Create the interface for the serving function with the raw features
    raw_features = tf.estimator.export.build_parsing_serving_input_receiver_fn(
        raw_feature_spec)().features

    receiver_tensors = {
        feature: tf.placeholder(shape=[None], dtype=raw_features[feature].dtype) 
        for feature in raw_features
    }

    receiver_tensors_expanded = {
        tensor: tf.reshape(receiver_tensors[tensor], (-1, 1)) 
        for tensor in receiver_tensors
    }

    # Apply the transform function 
    transformed_features = transform_output.transform_raw_features(
        receiver_tensors_expanded)

    return tf.estimator.export.ServingInputReceiver(
        transformed_features, receiver_tensors)


# Create eval input function for model evaluation
def _eval_input_receiver_fn(transform_output, schema):
    
    raw_feature_spec = schema_utils.schema_as_feature_spec(schema).feature_spec
    serving_input_receiver = tf.estimator.export.build_parsing_serving_input_receiver_fn(
        raw_feature_spec, default_batch_size=None)()
    
    features = serving_input_receiver.features.copy()
    transformed_features = transform_output.transform_raw_features(features)

    features.update(transformed_features)
    
    return tfma.export.EvalInputReceiver(
        features=features,
        receiver_tensors=serving_input_receiver.receiver_tensors,
        labels=transformed_features[TARGET_FEATURE_NAME]
    )



# TFX will call this function
def trainer_fn(hparams, schema):
    
    print("TensorFlow Version:" + tf.__version__)

    hidden_units = [128, 64]
    batch_size = 40
    train_steps = hparams.train_steps
    eval_steps =  hparams.eval_steps
    model_dir = hparams.serving_model_dir
    
    hparams.hidden_units = hidden_units
    
    # Load TFT Transform Output
    transform_output = tft.TFTransformOutput(hparams.transform_output)
    
    # Create Train Sepc
    train_spec = tf.estimator.TrainSpec(
        input_fn = make_input_fn(
            hparams.train_files,
            transform_output,
            batch_size=batch_size,
            num_epochs=None,
            shuffle=True
      ),
      max_steps=train_steps
    )
    
    # Create Exportor with the serving receiver function
    serving_receiver_fn = lambda: _serving_input_receiver_fn(transform_output, schema)
    exporter = tf.estimator.FinalExporter('census', serving_receiver_fn)
    
    # Create Eval Sepc
    eval_spec = tf.estimator.EvalSpec(
        input_fn = make_input_fn(
            hparams.eval_files,
            transform_output,
            batch_size=batch_size
        ),
        exporters=[exporter],
        start_delay_secs=0,
        throttle_secs=0,
        steps=eval_steps,
        name='census-eval'
    )
    
    # Create Run Config
    run_config = tf.estimator.RunConfig(
        tf_random_seed=19831006,
        save_checkpoints_steps=200, 
        keep_checkpoint_max=3, 
        model_dir=model_dir,
        log_step_count_steps=10
    )
    
    # Create estimator
    estimator = create_estimator(hparams, transform_output, run_config)

    # Create an input receiver for TFMA processing
    eval_receiver_fn = lambda: _eval_input_receiver_fn(
        transform_output, schema)
    
    return {
        'estimator': estimator,
        'train_spec': train_spec,
        'eval_spec': eval_spec,
        'eval_input_receiver_fn': eval_receiver_fn
    }

Overwriting ../src/train.py


### 4.2 Train the model using the Trainer component
* **Inputs**: train module file with the **trainer_fn**, raw schema (Schema), and transform output (TransformGraph)
* **Outputs**: saved_model (Model)
* **Properties**: train and eval args

We need to upload the train.py module to Cloud Storage to be accessible by AI Platform Training

In [72]:
gcs_train_module_file = WORKSPACE + '/src/train.py'
!gsutil cp {_train_module_file} {gcs_train_module_file}

Copying file://../src/train.py [Content-Type=text/x-python]...
/ [1 files][  6.4 KiB/  6.4 KiB]                                                
Operation completed over 1 objects/6.4 KiB.                                      


In [96]:
%%writefile Dockerfile
FROM gcr.io/tfx-oss-public/tfx:0.15.0
RUN pip install -q tensorflow==1.15

Overwriting Dockerfile


In [97]:
!docker build . -t gcr.io/{PROJECT_ID}/tfx-tf1

Sending build context to Docker daemon  2.659MB
Step 1/2 : FROM gcr.io/tfx-oss-public/tfx:0.15.0
 ---> c51561cfbbbc
Step 2/2 : RUN pip install -q tensorflow==1.15
 ---> Running in b12150a17d2e
ERROR: tensorflow-serving-api 2.0.0 has requirement tensorflow~=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
Removing intermediate container b12150a17d2e
 ---> 99eca62789a3
Successfully built 99eca62789a3
Successfully tagged gcr.io/ksalama-cloudml/tfx-tf1:latest


In [98]:
!docker push gcr.io/ksalama-cloudml/tfx-tf1:latest

The push refers to repository [gcr.io/ksalama-cloudml/tfx-tf1]

a77e4c7a: Preparing 
6b7e2b93: Preparing 
81f2de24: Preparing 
5d3d4213: Preparing 
30629c02: Preparing 
b6d4e428: Preparing 
b5e8fca8: Preparing 
3e5d6fe0: Preparing 
fb2e7b74: Preparing 
69ee7207: Preparing 
f8c62487: Preparing 
77e4c7a: Pushing  1.293GB/1.771GBAPushing  789.6MB/1.771GB77e4c7a: Pushed   1.778GB/1.771GBlatest: digest: sha256:6c07a2d3a5b657a13c4cc283076536dead9ab08c2026d92cd9027ba1f97333a2 size: 2846


In [99]:
from tfx.extensions.google_cloud_ai_platform.trainer import executor as ai_platform_trainer_executor
from tfx.components.base import executor_spec

# Cloud AI Platform training args
ai_platform_training_args = {
    'scaleTier': 'BASIC',
    'project': PROJECT_ID,
    'region': REGION,
    'masterConfig': {
        'imageUri': "gcr.io/{}/tfx-tf1:latest".format(PROJECT_ID)
        
    }
}

trainer = tfx.components.Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(
        ai_platform_trainer_executor.Executor),
    module_file=gcs_train_module_file,
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_importer.outputs['result'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=tfx.proto.trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=tfx.proto.trainer_pb2.EvalArgs(num_steps=None),
    custom_config={'ai_platform_training_args': ai_platform_training_args},
    instance_name='Census_Classifier_Trainer'
)

context.run(trainer)

/Users/khalidsalama/Technology/GoogleCloud/source-code/tfx-playground/venv/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


ExecutionResult(
    component_id: Trainer.Census_Classifier_Trainer
    execution_id: 35
    outputs:
        output: Channel(
            type_name: ModelExportPath
            artifacts: [Artifact(type_name: ModelExportPath, uri: gs://ksalama-cloudml-tfx/census/Trainer.Census_Classifier_Trainer/output/35/, split: , id: 15)]
        ))

In [100]:
train_uri = trainer.outputs['model'].get()[0].uri
serving_model_path = os.path.join(train_uri, 'serving_model_dir', 'export', 'census')
latest_serving_model_path = os.path.join(serving_model_path, max(gfile.listdir(serving_model_path)))
print(latest_serving_model_path)

gs://ksalama-cloudml-tfx/census/Trainer.Census_Classifier_Trainer/output/35/serving_model_dir/export/census/1574021639/


## 5. Evaluate the trained model (Evaluator)
* **Inputs**: eval data (Examples), trained model (Model)
* **Outputs** eval metric (ModelEvaluation)
* **Properties**: Slicing Specs

In [102]:
feature_slicing_spec=tfx.proto.evaluator_pb2.FeatureSlicingSpec(
    specs=[
        tfx.proto.evaluator_pb2.SingleSlicingSpec(),
        tfx.proto.evaluator_pb2.SingleSlicingSpec(column_for_slicing=['occupation'])
    ]
)


model_analyzer = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    feature_slicing_spec=feature_slicing_spec,
    instance_name="Occupation_based_Evaluator",
)

context.run(
    model_analyzer, 
    #beam_pipeline_args=df_beam_pipeline_args
)


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from gs://ksalama-cloudml-tfx/census/Trainer.Census_Classifier_Trainer/output/35/eval_model_dir/1574021687/variables/variables
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.get_tensor_from_tensor_info or tf.compat.v1.saved_model.get_tensor_from_tensor_info.


ExecutionResult(
    component_id: Evaluator.Occupation_based_Evaluator
    execution_id: 37
    outputs:
        output: Channel(
            type_name: ModelEvalPath
            artifacts: [Artifact(type_name: ModelEvalPath, uri: gs://ksalama-cloudml-tfx/census/Evaluator.Occupation_based_Evaluator/output/37/, split: , id: 16)]
        ))

In [103]:
import tensorflow_model_analysis as tfma

results_uri = model_analyzer.outputs['output'].get()[0].uri
eval_result = tfma.load_eval_result(results_uri)
eval_result.slicing_metrics

[((),
  {'': {'': {'auc': {'doubleValue': 0.9173318147659302},
     'auc_precision_recall': {'doubleValue': 0.7917448282241821},
     'accuracy': {'doubleValue': 0.8612532615661621},
     'average_loss': {'doubleValue': 0.29889586567878723},
     'recall': {'doubleValue': 0.6070519089698792},
     'label/mean': {'doubleValue': 0.23663946986198425},
     'post_export_metrics/example_count': {'doubleValue': 9545.0},
     'precision': {'doubleValue': 0.7584112286567688},
     'prediction/mean': {'doubleValue': 0.23555438220500946},
     'accuracy_baseline': {'doubleValue': 0.7633605003356934}}}}),
 ((('occupation', ' Other-service'),),
  {'': {'': {'label/mean': {'doubleValue': 0.024558069184422493},
     'post_export_metrics/example_count': {'doubleValue': 936.0},
     'precision': {'doubleValue': 0.5508597493171692},
     'accuracy_baseline': {'doubleValue': 0.9754419326782227},
     'prediction/mean': {'doubleValue': 0.03925583139061928},
     'auc': {'doubleValue': 0.8923314809799194}

In [104]:
tfma.view.render_slicing_metrics(
    result=eval_result, 
    slicing_column='occupation'
)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'post_export_metrics/example_count'}, data=[{'slice': '…

## 6. Validate the Trained Model

We will create a Custom TFX Component that validates the trained model based on its produced evaluation metric.

The custom validator will **bless** the model if:
1. Overal accuracy is greater than 85%.
2. Accuracy per **Occupation** slice is at most 10% less than the overall accuracy.

* **Inputs**: Evaluation Metric (ModelEvaluation), trained model (Model)
* **Outputs**: blessing (ModelBlessing)
* **Properties**: accuracy_threshold, slice_accuracy_tolerance

### 6.1. Create ComponentSepc

In [106]:
from tfx.types import standard_artifacts
from tfx.types.component_spec import ChannelParameter
from tfx.types.component_spec import ExecutionParameter

class AccuracyValidatorSpec(tfx.types.ComponentSpec):
    
    INPUTS = {
        'eval_results': ChannelParameter(type=standard_artifacts.ModelEvaluation),
        'model': ChannelParameter(type=standard_artifacts.Model),
    }
    
    OUTPUTS = {
      'blessing': ChannelParameter(type=standard_artifacts.ModelBlessing),
    }
    
    PARAMETERS = {
        'accuracy_threshold': ExecutionParameter(type=float),
        'slice_accuracy_tolerance': ExecutionParameter(type=float),
    }


## 6.2. Create Custom Exectutor

In [107]:
from tfx.components.base import base_executor
from tfx.types import artifact_utils
from tfx.utils import io_utils

class AccuracyValidatorExecutor(base_executor.BaseExecutor):
    
    def Do(self, input_dict, output_dict, exec_properties):
        
        valid = True
        
        self._log_startup(input_dict, output_dict, exec_properties)
        
        accuracy_threshold = exec_properties['accuracy_threshold']
        slice_accuracy_tolerance = exec_properties['slice_accuracy_tolerance']
        min_slice_accuracy = accuracy_threshold - slice_accuracy_tolerance
        print("Accuracy Threshold:", accuracy_threshold)
        print("Slice Accuracy Tolerance:", slice_accuracy_tolerance)
        print("Min Accuracy per Slice:", min_slice_accuracy)
        
        results_uri = input_dict['eval_results'][0].uri
        eval_results = tfma.load_eval_result(results_uri)
        
        overall_acc = eval_result.slicing_metrics[0][1]['']['']['accuracy']['doubleValue']
        print("Overall accuracy:", overall_acc)
        
        if overall_acc >= accuracy_threshold:
            for slicing_metric in eval_result.slicing_metrics:
                slice_acc = slicing_metric[1]['']['']['accuracy']['doubleValue']
                if slice_acc < min_slice_accuracy:
                    print("Slice accuracy value < min accuracy:", slice_acc )
                    valid = False
                    break
        else:
            valid = False
        
        print("Valid:", valid)
        
        blessing = artifact_utils.get_single_instance(output_dict['blessing'])
        
        # Current model.
        current_model = artifact_utils.get_single_instance(input_dict['model'])
        blessing.set_string_custom_property('current_model', current_model.uri)
        blessing.set_int_custom_property('current_model_id', current_model.id)

        if valid:
            io_utils.write_string_file(os.path.join(blessing.uri, 'BLESSED'), '')
            blessing.set_int_custom_property('blessed', 1)
        else:
            io_utils.write_string_file(os.path.join(blessing.uri, 'NOT_BLESSED'), '')
            blessing.set_int_custom_property('blessed', 0)


### 6.3. Create AccuracyModelValidator

In [108]:
from typing import Optional
from tfx import types
from tfx.components.base import base_component
from tfx.components.base import executor_spec

class AccuracyModelValidator(base_component.BaseComponent):

    SPEC_CLASS = AccuracyValidatorSpec
    EXECUTOR_SPEC = executor_spec.ExecutorClassSpec(AccuracyValidatorExecutor)
    
    def __init__(self,
                 eval_results: types.channel,
                 model: types.channel,
                 accuracy_threshold: float,
                 slice_accuracy_tolerance: float,
                 blessing: Optional[types.Channel] = None,
                 instance_name=None):
        
        blessing = blessing or types.Channel(
            type=standard_artifacts.ModelBlessing,
            artifacts=[standard_artifacts.ModelBlessing()])
        
        spec = AccuracyValidatorSpec(
            eval_results=eval_results, model=model, blessing=blessing, 
            accuracy_threshold=accuracy_threshold, slice_accuracy_tolerance=slice_accuracy_tolerance
        )
        
        super().__init__(spec=spec, instance_name=instance_name)
    

In [109]:
accuracy_model_validator = AccuracyModelValidator(
    eval_results=model_analyzer.outputs['output'],
    model=trainer.outputs['model'],
    accuracy_threshold=0.85,
    slice_accuracy_tolerance=0.15,
    instance_name="Accuracy_Model_Validator"
)

context.run(accuracy_model_validator)

Accuracy Threshold: 0.85
Slice Accuracy Tolerance: 0.15
Min Accuracy per Slice: 0.7
Overall accuracy: 0.8612532615661621
Valid: True


ExecutionResult(
    component_id: AccuracyModelValidator.Accuracy_Model_Validator
    execution_id: 39
    outputs:
        blessing: Channel(
            type_name: ModelBlessingPath
            artifacts: [Artifact(type_name: ModelBlessingPath, uri: gs://ksalama-cloudml-tfx/census/AccuracyModelValidator.Accuracy_Model_Validator/blessing/39/, split: , id: 17)]
        ))

In [112]:
blessing_uri = accuracy_model_validator.outputs.blessing.get()[0].uri
!gsutil ls {blessing_uri}

gs://ksalama-cloudml-tfx/census/AccuracyModelValidator.Accuracy_Model_Validator/blessing/39/
gs://ksalama-cloudml-tfx/census/AccuracyModelValidator.Accuracy_Model_Validator/blessing/39/BLESSED


## 7. Pushing the Blessed Model (Pusher)
This steps pushes the validated and blessed model to its final destination. This could be:
1. A Model Registry
2. Git repository
3. API Serving Platform
4. A specific filesystem location
5. ...

### 7.1. Push the blessed model to AI Platform Prediction

In [117]:
MODEL_NAME = 'census_classifier'

ai_platform_serving_args = {
    'project_id': PROJECT_ID,
    'model_name': MODEL_NAME
}

In [118]:
from tfx.components.base import executor_spec
from tfx.extensions.google_cloud_ai_platform.pusher import executor as ai_platform_pusher_executor 

pusher = tfx.components.Pusher(
      custom_executor_spec=executor_spec.ExecutorClassSpec(
         ai_platform_pusher_executor.Executor),
      model_export=trainer.outputs.model,
      model_blessing=accuracy_model_validator.outputs.blessing,
      custom_config={'ai_platform_serving_args': ai_platform_serving_args})

context.run(pusher)

/Users/khalidsalama/Technology/GoogleCloud/source-code/tfx-playground/venv/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


ExecutionResult(
    component_id: Pusher
    execution_id: 41
    outputs:
        model_push: Channel(
            type_name: ModelPushPath
            artifacts: [Artifact(type_name: ModelPushPath, uri: gs://ksalama-cloudml-tfx/census/Pusher/model_push/41/, split: , id: 18)]
        ))

### 7.2. Test the Served model

In [121]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
credentials = GoogleCredentials.get_application_default()

service = discovery.build('ml', 'v1', credentials=credentials)
model_url = 'projects/{}/models/{}'.format(PROJECT_ID, MODEL_NAME)

def census_classifier(instances):
    request_data = {
        'instances': instances
    }

    response = service.projects().predict(
        body=request_data,
        name=model_url
    ).execute()

    output = response
    return output

In [126]:
output = census_classifier(
    [
        {
            'age': 34,
            'workclass': 'Private',
            'education': 'Doctorate',
            'education_num': 10,
            'marital_status': 'Married-civ-spouse',
            'occupation': 'Prof-specialty',
            'relationship': 'Husband',
            'race': 'White',
            'gender': 'Male',
            'capital_gain': 0, 
            'capital_loss': 0, 
            'hours_per_week': 40,
            'native_country':'Egyptian'
        }
    ]
)

print(output)

{'predictions': [{'classes': [' <=50K'], 'logistic': [0.24225106835365295], 'all_classes': [' <=50K', ' >50K'], 'class_ids': [0], 'probabilities': [0.7577489018440247, 0.24225103855133057], 'all_class_ids': [0, 1], 'logits': [-1.1403775215148926]}]}


## 8. Querying Metadata database

In [128]:
!pip install -U -q mysql-connector

In [138]:
import mysql.connector

metadb = mysql.connector.connect(
    host='127.0.0.1',
    port=3306,
    database='metadb',
    user="root",
    passwd="root"
)

cursor = metadb.cursor()

### Query Artifact table

In [139]:
cursor.execute("SELECT * FROM Artifact;")
pprint(cursor.fetchall())

[(1, 1, '../data/train'),
 (2,
  4,
  'gs://ksalama-cloudml-tfx/census/CsvExampleGen.Data_Extraction_Spliting/examples/5/train/'),
 (3,
  4,
  'gs://ksalama-cloudml-tfx/census/CsvExampleGen.Data_Extraction_Spliting/examples/5/eval/'),
 (4,
  4,
  'gs://ksalama-cloudml-tfx/census/BigQueryExampleGen.BQ_Data_Extraction_Spliting/examples/9/train/'),
 (5,
  4,
  'gs://ksalama-cloudml-tfx/census/BigQueryExampleGen.BQ_Data_Extraction_Spliting/examples/9/eval/'),
 (6,
  7,
  'gs://ksalama-cloudml-tfx/census/StatisticsGen.Statistics_Generation/output/12/train/'),
 (7,
  7,
  'gs://ksalama-cloudml-tfx/census/StatisticsGen.Statistics_Generation/output/12/eval/'),
 (8, 8, '../raw_schema'),
 (9,
  11,
  'gs://ksalama-cloudml-tfx/census/ExampleValidator.Data_Validation/output/14/'),
 (10,
  13,
  'gs://ksalama-cloudml-tfx/census/Transform.Data_Transformation/transform_output/18/'),
 (11,
  4,
  'gs://ksalama-cloudml-tfx/census/Transform.Data_Transformation/transformed_examples/18/train/'),
 (12,
  4